In [1]:

import numpy as np
import pandas as pd
from pandas import DataFrame
from datetime import datetime

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import chart_studio.plotly as py

from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics

from sklearn.preprocessing import MinMaxScaler
import pmdarima as pm

%matplotlib inline
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


import ipywidgets as widgets
from ipywidgets import interact,Layout
import io

from IPython.display import display,HTML,Javascript, Markdown as md

from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings("ignore")

In [2]:


uploader = widgets.FileUpload(
           accept='*.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
           multiple=False  # True to accept multiple files upload else False
)


In [3]:
display(HTML('<h1 align="center">Time Series Forecasting</h1>'))
display(HTML('<h2>Upload the CSV file below</h2>'))

In [4]:
    @interact(val=uploader)
    def fbprophet(val):

        if val=={}: return None
        input_file = list(uploader.value.values())[0]

        print("The Uploaded file name is ",input_file['metadata']['name'])
        content = input_file['content']
        content = io.StringIO(content.decode('utf-8'))
        dataset = pd.read_csv(content)
        display(HTML('<h2>Statistical Analysis of Close Price attribute</h2>'))
 
        try:
            datetime.strptime(dataset['Date'][1], '%Y-%m-%d') 
            dataset['Date']=pd.to_datetime(dataset['Date'],format='%Y/%m/%d')
        except ValueError:
            dataset['Date']=pd.to_datetime(dataset['Date'],format='%d/%m/%Y')

        dataset.set_index('Date',inplace=True)
        print(dataset['Close'].describe())
        dataset.dropna(inplace=True)

        display(HTML('<hr>'))
        fig=px.line(dataset,x=dataset.index,y=dataset['Close'],title='Original Data Graph of Close Price')
        fig.show()

        dataset['Close:Rolling 30 days']=dataset['Close'].rolling(30).mean().dropna()
        dataset['Close:Rolling 365 days']=dataset['Close'].rolling(365).mean().dropna()    

        fig1 = make_subplots(
              rows=1, cols=2,
              subplot_titles=("Monthly Rolling Mean", "Yearly Rolling Mean"))

        fig1.add_trace(go.Scatter(x=dataset.index, y=dataset['Close'],mode='lines'),
                  row=1, col=1)
        fig1.add_trace(go.Scatter(x=dataset.index, y=dataset['Close'],mode='lines'),
                  row=1, col=2)

        fig1.add_trace(go.Scatter(x=dataset.index, y=dataset['Close:Rolling 30 days'],mode='lines',name='Monthly mean'),
                  row=1, col=1) 
        fig1.add_trace(go.Scatter(x=dataset.index, y=dataset['Close:Rolling 365 days'],mode='lines',name='Yearly mean'),
                  row=1, col=2)

        fig1.update_xaxes(title_text="Date", row=1, col=1)
        fig1.update_xaxes(title_text="Date", row=1, col=2)


        # Update yaxis properties
        fig1.update_yaxes(title_text="Close Price:Rolling 30 days", row=1, col=1)
        fig1.update_yaxes(title_text="Close Price:Rolling 365 days", row=1, col=2)

        fig1.update_layout(height=500,width=1400)
        fig1.show()    

        display(HTML('<hr>'))

        #divide into train and validation set
        train = dataset[:int(0.8*(len(dataset)))]
        test = dataset[int(0.8*(len(dataset))):]
        
        #divide into train and test set for FBprophet
        fbtrain = dataset[:int(len(dataset))-365]
        fbtest = dataset[int(len(dataset))-365:]

        display(HTML('<h2>Processing Models......</h2>'))
        
        #Auto arima
        model3 = pm.auto_arima(train['Close'], start_p=0, start_q=0,
                          test='adf',       # use adftest to find optimal 'd'
                          max_p=6, max_q=6, # maximum p and q
                          #m=1,              # frequency of series
                          d=None,           # let model determine 'd'
                          seasonal=False,    # No Seasonality
                          start_P=0,
                          scoring='mae',  
                          D=0, 
                          trace=True,
                          error_action='ignore',
                          suppress_warnings=True,
                          stepwise=False)


        #SARIMA
        model4 = pm.auto_arima(train['Close'], start_p=0, start_q=0,
                         test='adf',       # use adftest to find optimal 'd'
                          max_p=3, max_q=3, # maximum p and q
                          m=12,              # frequency of series
                          d=None,           # let model determine 'd'
                          seasonal=True,    # No Seasonality
                          max_P=3,max_Q=3,
                          scoring='mae',
                          D=None,
                          trace=True,
                          error_action='ignore',
                          suppress_warnings=True,
                          stepwise=True)


        display(HTML('<hr>'))

        display(HTML('<h2>ARIMA Model Summary Results</h2>'))
        print(model3.summary())
        

        #forecasting dataset
        forecast = model3.predict(n_periods=len(test))
        arima_past_predicted=model3.predict_in_sample(start=1,end=len(train))
        sarima_past_predicted=model4.predict_in_sample(start=1,end=len(train))
        forecast = pd.DataFrame(forecast,index = test.index,columns=['Prediction'])

        #forecasting dataset
        forecast1 = model4.predict(n_periods=len(test))
        forecast1 = pd.DataFrame(forecast1,index = test.index,columns=['Prediction'])

        #plotting
        display(HTML('<hr>'))

        fig2 = make_subplots(
              rows=1, cols=2,
              subplot_titles=("ARIMA Model", "Seasonal ARIMA Model"))
        fig2.add_trace(go.Scatter(x=train.index, y=train['Close'],name='Train',mode='lines'),
                  row=1, col=1)

        fig2.add_trace(go.Scatter(x=test.index, y=test['Close'],name='Test',mode='lines'),
                  row=1, col=1)

        fig2.add_trace(go.Scatter(x=forecast.index, y=forecast['Prediction'],name='Predicted',mode='lines'),
                  row=1, col=1)

        fig2.add_trace(go.Scatter(x=train.index, y=train['Close'],name='Train',mode='lines'),
                 row=1, col=2)

        fig2.add_trace(go.Scatter(x=test.index, y=test['Close'],name='Test',mode='lines'),
                  row=1, col=2)

        fig2.add_trace(go.Scatter(x=forecast1.index, y=forecast1['Prediction'],name='Predicted',mode='lines'),
                  row=1, col=2)    

        # Update xaxis properties
        fig2.update_xaxes(title_text="Date", row=1, col=1)
        fig2.update_xaxes(title_text="Date", row=1, col=2)


        # Update yaxis properties
        fig2.update_yaxes(title_text="Stocks Close Price", row=1, col=1)
        fig2.update_yaxes(title_text="Stocks Close Price", row=1, col=2)

        fig2.update_layout(height=500,width=1400)
        fig2.show()

        display(HTML('<hr>'))

        display(HTML('<h2>Seasonal ARIMA Model Summary Results</h2>'))
        print(model4.summary())

        display(HTML('<hr>'))

        dataset.reset_index('Date',inplace=True)
        fbtrain.reset_index('Date',inplace=True)
        columns =['Date','Close']
        ndf=pd.DataFrame(fbtrain,columns=columns)

        prophet_df=ndf.rename(columns={'Date':'ds','Close':'y'})

        m=Prophet()
        m.fit(prophet_df)

        future=m.make_future_dataframe(periods=len(fbtest),freq='B')
        forecast_fb=m.predict(future)
        forecast_fb['y']=prophet_df['y']


        ndf1=pd.DataFrame(dataset,columns=columns)

        prophet_df1=ndf1.rename(columns={'Date':'ds','Close':'y'})

        m1=Prophet()
        m1.fit(prophet_df1)

        future1=m1.make_future_dataframe(periods=len(fbtest), freq='B')
        forecast_fb1=m1.predict(future1)
        forecast_fb1['y']=prophet_df1['y']    



        fig3 = make_subplots(
              rows=1, cols=2,
              subplot_titles=("Future forecast", "Future forecast with range "))
        fig3.add_trace(go.Scatter(x=forecast_fb['ds'], y=forecast_fb['y'],mode='lines',name='Train'),
                  row=1, col=1)
        fig3.add_trace(go.Scatter(x=fbtest.index, y=fbtest['Close'],name='Test',mode='lines'),
                  row=1, col=1)    
        fig3.add_trace(go.Scatter(x=forecast_fb['ds'],y=forecast_fb['yhat'],mode='lines',name='Predicted'),
                  row=1, col=1)

        fig3.add_trace(go.Scatter(x=forecast_fb1['ds'], y=forecast_fb1['y'],mode='lines',name='Observed',line={'color':'blue'}),
                  row=1, col=2)
        fig3.add_trace(go.Scatter(x=forecast_fb1['ds'],y=forecast_fb1['yhat'],mode='lines',name='Predicted',line={'color':'red'}),
                  row=1, col=2)

        fig3.add_trace(go.Scatter(x=forecast_fb1['ds'][len(forecast_fb1)-365:len(forecast_fb1)],
                                  y=forecast_fb1['yhat_upper'][len(forecast_fb1)-365:len(forecast_fb1)],
                                  mode='lines',name='Upper Range',marker=dict(color="#444"),line=dict(width=0),
                                 showlegend=False),
                                #line={'color':'orange'}),
                  row=1, col=2)      

        fig3.add_trace(go.Scatter(x=forecast_fb1['ds'][len(forecast_fb1)-365:len(forecast_fb1)],
                                  y=forecast_fb1['yhat_lower'][len(forecast_fb1)-365:len(forecast_fb1)],
                                  mode='lines',name='Lower range',marker=dict(color="#444"),line=dict(width=0),
                                 fillcolor='rgba(68, 68, 68, 0.3)',
                                 fill='tonexty',showlegend=False),
                                  #,line={'color':'green'}),
                  row=1, col=2)

        # Update xaxis properties
        fig3.update_xaxes(title_text="Date", row=1, col=1)
        fig3.update_xaxes(title_text="Date", row=1, col=2)


        # Update yaxis properties
        fig3.update_yaxes(title_text="Stocks Close Price", row=1, col=1)
        fig3.update_yaxes(title_text="Stocks Close Price", row=1, col=2)    

        fig3.update_layout(height=500,width=1400,title_text='Facebook Prophet Model')
        fig3.show()

        display(HTML('<hr>'))

        display(HTML('<h2>Performance of model using R2 score</h2>'))
        
        #Performance of the Model using R2 score
        
        arima_train_predicted=pd.Series(arima_past_predicted)
        sarima_train_predicted=pd.Series(sarima_past_predicted)
        
        Arima_model_rscore=r2_score(dataset['Close'],arima_train_predicted.append(forecast['Prediction']))
        print('ARIMA Model : ',Arima_model_rscore)

        Sarima_model_rscore=r2_score(dataset['Close'],sarima_train_predicted.append(forecast1['Prediction']))
        print('SARIMA Model : ',Sarima_model_rscore)

        fbprophet_model_rscore=r2_score(dataset['Close'],forecast_fb['yhat'])
        print('Facebook Prophet Model : ',fbprophet_model_rscore)






interactive(children=(FileUpload(value={}, accept='*.csv', description='Upload'), Output()), _dom_classes=('wi…